# TP2- Introducción al SGBD MongoDB y Conexión a la API Rest de Twitter

En este trabajo práctico queremos descubrir las características del SGBD no relacional MongoDB. En este sentido, vamos a desarrollar un script que se conecta a la API Rest de Twitter para: 1) descargar datos al formato JSON, 2) almacenarlos en una base Mongo y 3) realizar algunas consultas sobre los datos.

## 1. Conectarse a la API Rest de Twitter

- ¿Cómo crear una aplicación utilizando los datos de Twitter?: https://developer.twitter.com/en/docs/basics/getting-started
- Crear una aplicación utilizando los datos de Twitter: https://apps.twitter.com/
- ¿Qué deberían saber todos los desarrolladores antes de jugar con la API de twitter?: https://developer.twitter.com/en/docs/basics/things-every-developer-should-know

In [1]:
from twython import Twython

APP_KEY = 'eGeBZaHYfDSWRwuoVkLso5de4'
APP_SECRET = 'nbsZSGg90jLvtc6DxyK2PqvIDDI0h8gQ3pCfuYuJo1b8yHL8F1'
OAUTH_TOKEN = '250409598-7Ca7Tv90s1u4mbiIuurKskryClS1WcnGma9unW16'
OAUTH_TOKEN_SECRET = 'p606FeOvl2UuwMq1FykXtnn3EAUVWsmELr6xChu4jBbpd'

twitter = Twython(APP_KEY, APP_SECRET,OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

## 2. Consultar la API de Twitter

- Utilizando una librería Python que permite enviar consultas HTTP a la API de Twitter, conectarse a un 'endpoint' de la API para leer datos. Ver la lista de los endpoints: https://developer.twitter.com/en/docs/tweets/post-and-engage/overview

In [2]:
A=twitter.search(q='valdivia',count=100)

for i in A["statuses"]:
    print(i["text"])
    print("///////////////")
    t = {
        'created_at':i['created_at'],
        'text':i['text'],
        'name':i['user']['name'],
        'description':i['user']['description'],
        'location':i['user']['location']
    }

@Cooperativa @AldoRomuloS @marcelobarti @polaco_goldberg Me parece que CC puede ganar el clásico, mal y todo como a… https://t.co/OGPwdSLrQk
///////////////
@ColoColo Vamos que hoy dia se gana mierda!!! Hoy tienen que aparecer con mas fuerza los Valdivia, Valdes y Paredes… https://t.co/VS2YSgW41X
///////////////
@jmc_valdivia Que suelto de cuerpo, Teilier para mentir, justificando a su sobrino, q fue detenido en Temuco, con p… https://t.co/pnXYlrlvnQ
///////////////
RT @FcoDarmendral: Día sombrío en el río Valdivia tras el terremoto de 1960. Fotografía de Mario Guillard. https://t.co/YNTQhVjhiq
///////////////
@LucasGomes9922 @Alanziitoooo @SaoPauloFC Tlgd , agr o São Paulo tá necessitando d um atacante estilo ele , Marcos… https://t.co/KkHELcu4SI
///////////////
@Duran_spfc @ELSanPabloFC @SaoPauloFC Sidão, Militão, arboleda, R. Caio, Regis e Jr tavares, lizieiro, Petros, juci… https://t.co/ZKDY3iAYll
///////////////
RT @lectorcl: ¡Hoy! Desde las 10.30 a las 14.30 horas Libros sin IVA 

## 3. Conectarse al SGBD Mongo

- Conectarse a Mongo desde Python utilizando la librería adecuada

In [3]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient()
db = client.test

## 4. Escribir datos de Twitter en una Base de Datos Mongo

In [4]:
A=twitter.search(q='@NintendoAmerica', result_type='popular',count=15)

for i in A["statuses"]:
    t = {
        'created_at':i['created_at'],
        'text':i['text'],
        'name':i['user']['name'],
        'description':i['user']['description'],
        'location':i['user']['location']
    }
    db.tweets.insert_one(t)

- #### mostrar por lugar

In [5]:
result = db.tweets.find({})
for tweet in result:
    print(tweet['location'])



Uppsala, Sweden
England, United Kingdom

Boston, MA, USA
Bordeaux, France
Ontario, Canada
Uppsala, Sweden
Milwaukie, Oregon
Austin, TX
New York, USA


Uppsala, Sweden

Bordeaux, France
Milwaukie, Oregon
England, United Kingdom


Uppsala, Sweden

Bordeaux, France
Milwaukie, Oregon
England, United Kingdom


Uppsala, Sweden

Bordeaux, France
Milwaukie, Oregon
England, United Kingdom


- #### tweets que contienen la palabra "UACh"

In [6]:
result = db.tweets.find({'tweet_text':{'$regex':'UACh',"$options":"$i"}})
print(result.count())
for tweet in result:
    print(tweet['tweet_text'])

0


- #### contar tweets por usuario

In [7]:
result = db.tweets.aggregate([{"$group":{"_id":"$name","total":{"$sum":1}}}])
for tweet in result:
    print(tweet)

{'total': 1, '_id': 'Tech Insider'}
{'total': 4, '_id': 'Liam Robertson'}
{'total': 4, '_id': 'Motion Twin'}
{'total': 5, '_id': 'Wolfenstein'}
{'total': 4, '_id': 'Dark Souls'}
{'total': 1, '_id': 'Harmonix'}
{'total': 4, '_id': 'Crash Bandicoot'}
{'total': 4, '_id': 'Dark Horse Comics'}
{'total': 4, '_id': 'Hasbro'}
{'total': 1, '_id': '#DESTROYERS 🇨🇦'}
{'total': 1, '_id': 'Jools Watsham'}


#### Parte 2

#### 1) En una sola consulta, actualizar los documentos tweets que hablan de "valdiviacl", agregando un campo "city" con el valor siguiente: { type: "Point", coordinates: [-73.245209,-39.819588] }

In [16]:
db.tweets.update_many( {'tweet_text':{'$regex':'valdiviacl', "$options":"$i"}},
                      {'$set': {'city':{'type':'Point','coordinates':[-73.245209,-39.819588]}}})

#### 2) Recopilar algunos tweets hablando de "Puerto Montt" y agregar un campo "city" con las coordinades de Puerto Montt

In [17]:
db.tweets.update_many( {'tweet_text':{'$regex':'Puerto Montt', "$options":"$i"}}, 
                      {'$set': {'city':{'type':'Point','coordinates':[-41.462561,-72.9988162]}}})

#### 3) Utilizar un operator de Mongo para recopilar el documento lo más cercano del punto: -73.131920, -40.574505

In [19]:
B=db.tweets.find( {
    'location': {
        '$near': {
            '$geometry': {'type': "Point" ,'coordinates': [-73.131920, -40.574505 ]},
            '$maxDistance': 5000,
            '$minDistance': 1000
        }
    }
})
print(B)